#### Assume that the claims count random variable N has a Poisson distribution with given years at risk v > 0 and expected frequency λ > 0. We aim at modeling the expected frequency λ > 0 such that it allows us to incorporate structural differences (heterogene-ity) between different insurance policies and risks; such structural differences are called systematic effects in the statistical literature.

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [15]:
claimsdf = pd.read_csv('/home/julian/Cursos/Ironhack/Proyectos/ProyectoFinal/Claims-Frequency-Predictions/Notebooks/claimsdf_1.csv')

In [16]:
claimsdf.head()

,ClaimNb,Exposure,Area,BonusMalus,VehBrand,Density,Region,empirical_frequencies,VehGas_Regular,VehPower_,VehAge_,DrivAge_
0,1,0.10,4,50,9,1217,1.0,10.000000,1,5,1,6
1,1,0.77,4,50,9,1217,1.0,1.298701,1,5,1,6
2,1,0.75,2,50,9,54,5.0,1.333333,0,6,2,6
3,1,0.09,2,50,9,76,7.0,11.111111,0,7,1,5
4,1,0.84,2,50,9,76,7.0,1.190476,0,7,1,5


## TAKE OUT EMP FREQ, AGRUPAR REGIONES Y HACER HOT ENCODING

In [17]:
claimsdf.drop(columns="empirical_frequencies", inplace=True)

### GENERALIZED LINEAR MODELS (Poisson case)

- The feature components interact in a multiplicative way in our Poisson GLM. One of the main tasks is to analyze whether this multiplicative interaction is appropriate.
- A we have 4 categorical classes, then we need 3 indicators (dummy variables) to uniquely identify the parametrization of the (multiplicative) model including an intercept.
- We consider 3 continuous feature components (Area, BonusMalus, log-Density), 1 binary feature component (VehGas) and 5 categorical feature components (VehPower, VehAge, DrivAge, VehBrand, Region)
- If we consider the 5 continuous feature components VehAge, DrivAge, VehPower, Area and log_density as log-linear, VehGas as a binary component (0 or 1), and the 2 categorical components VehBrand and Region by dummy coding we get a feature space dimension q = 5 + 1 + 10 + 12 = 28.

### MODEL TRAINING

- TRAIN SAMPLE, TEST SAMPLE

- Maximizing the log-likelihood for parameter β is equivalent to minimizing the deviance loss for β. In this spirit, the deviance loss plays the role of the canonical objective function that should be minimized. b is the scaled Poisson deviance loss. 

### OVER DISPERSION CHECK

- Deviance statistics that accounts for potential over- or under-dispersion φ != 1. In the Poisson model this does not apply because by definition φ = 1 (variance equal to mean). We can determine this dispersion parameter empirically on our data by Pearson’s (distribution-free) dispersion estimate and and the deviance dispersion

### MODEL SELECTION

- The likelihood ratio test based on Posisson deviance can be applied recursively to a sequence of nested models. This leads to a step-wise reduction of model complexity, this is similar in spirit to the analysis of variance (ANOVA) in Listing 2.7, and it is often referred to as backward model selection